In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# LDDMM landmark stochastics and (possible) collision

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from src.manifolds.landmarks import *
M = landmarks(4)
print(M)
from src.plotting import *

from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

In [ ]:
# example configuration
M.k_sigma = jnp.diag(jnp.array([.5,.5]))
n_steps = 5000

## Brownian Motion

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),np.zeros(M.N))).T.flatten())

(ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=n_steps))

# plot
M.newfig()
M.plot()
M.plot_path(zip(xs,charts))
plt.show()

# plot multiple sample paths
N = 5
xss = np.zeros((N,n_steps,M.dim))
chartss = np.zeros((N,xs.shape[0],q[1].shape[0]))
for i in range(N):
    (ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=n_steps))
    xss[i] = xs
    chartss[i] = charts

# plot
M.newfig()
M.plot()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, N)]
for i in range(N):
    M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
M.plotx(q,color='r')
plt.show()

# Simulations in various dimensions

In [ ]:
def simBrownianN2(d,N):
    M.setdim(d,jnp.diag(jnp.array([.5]*d)))
    M.setN(2)

    q = M.coords(jnp.vstack((np.linspace(-.5,.5,M.N),jnp.zeros((d-1,M.N)))).T.flatten())

    # generate multiple sample paths
    xss = np.zeros((N,n_steps,M.dim))
    chartss = np.zeros((N,n_steps,q[1].shape[0]))
    for i in range(N):
        (ts,xs,charts) = M.Brownian_coords(q,dWs(M.dim,n_steps=n_steps))
        xss[i] = xs
        chartss[i] = charts

    # plot
    colormap = plt.get_cmap('winter')
    colors=[colormap(k) for k in np.linspace(0, 1, N)]
#     if d == 1:
#         M.newfig()
#         M.plot()
#         for i in range(N):
#             plt.plot(xss[i][:,0],xss[i][:,1],color=colors[i])
#         plt.plot(q[0][0],q[0][1],'o',color='r',markersize=10)
#         plt.show()
#     elif d == 2:
#         # plot
#         M.newfig()
#         M.plot()
#         for i in range(N):
#             M.plot_path(zip(xss[i],chartss[i]),color=colors[i])
#         M.plotx(q,color='r')
#         plt.show()
        
    minv,mini = jnp.inf,-1

    # plot distance function
    d2s = np.zeros((N,n_steps))
    for i in range(N):
        xs = xss[i].reshape((n_steps,M.N,M.m))
        d2 = jnp.sum((xs[:,0]-xs[:,1])**2,1)
        d2s[i] = d2
        if jnp.nanmin(d2) < minv:
            minv, mini = jnp.nanmin(d2), i
        plt.plot(jnp.cumsum(dts(n_steps=n_steps)),jnp.log(d2),color=colors[i])
    plt.xlabel('t'), plt.ylabel('log(distance^2)')
    plt.show()
    
    print("min distance^2,log(distance^2):",jnp.nanmin(d2s[mini]),jnp.nanmin(jnp.log(d2s[mini])))
    
    # finding percentiles
    pcts = (np.array([0, 20, 35, 45, 55, 65, 80, 99])/100*N).astype('int')
    layers = np.empty([n_steps, 8])
    for i in range(n_steps) : 
        _sorted = np.sort(jnp.log(d2s[:,i]))
        layers[i, :] = _sorted[pcts]

    # plot the layers
    for i in range(7) :
        plt.fill_between(jnp.cumsum(dts(n_steps=n_steps)), layers[:, i], layers[:, i+1], color=["yellow", "blue", "green", "red", "green", "blue", "yellow"][i])
    plt.xlabel('t'), plt.ylabel('log(distance^2)')
    plt.show()

    # plot paths with closest endpoints
    lastj = -1 # last non-nan entry
    for j in range(n_steps):
        if jnp.any(jnp.isnan(xss[mini][j])):
            lastj = j-1
            break
    if d == 1:
        points = xss[mini].reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        M.newfig()
        M.plot()
        ax = plt.gca()

        norm = plt.Normalize(0,1)
        lc = LineCollection(segments, cmap='viridis', norm=norm)
        lc.set_array(jnp.cumsum(dts(n_steps=n_steps)))
        line = ax.add_collection(lc)

        plt.plot(q[0][0],q[0][1],'o',color='r',markersize=10)
        plt.plot(xss[mini][lastj][0],xss[mini][lastj][1],'o',color='k',markersize=10) 

        plt.gcf().colorbar(line, ax=ax)

        ax.set_xlim(xss[mini][:,0].min()-.5, xss[mini][:,0].max()+.5)
        ax.set_ylim(xss[mini][:,1].min()-.5, xss[mini][:,1].max()+.5)
        plt.show()
    elif d == 2:
        # plot
        M.newfig()
        M.plot()
        M.plot_path(zip(xss[mini],chartss[mini]),color=colors[mini])
        M.plotx(q,color='r')
        M.plotx((xss[mini][lastj],chartss[mini][lastj]),color='k')
        plt.show()
    elif d == 3:
        # plot
        M.newfig()
        M.plot()
        M.plot_path(zip(xss[mini],chartss[mini]),color=colors[mini])
        M.plotx(q,color='r')
        M.plotx((xss[mini][lastj],chartss[mini][lastj]),color='k')
        plt.show()


In [ ]:
N = 50

In [ ]:
# Gaussian kernel, R^1
M = landmarks(2)

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(1,N)

In [ ]:
# Exponential kernel, R^1
M = landmarks(2,kernel='K1')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(1,N)

In [ ]:
# Exponential kernel, R^1
M = landmarks(2,kernel='K2')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(1,N)

In [ ]:
# Exponential kernel, R^1
M = landmarks(2,kernel='K3')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(1,N)

In [ ]:
# Exponential kernel, R^1
M = landmarks(2,kernel='K4')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(1,N)

In [ ]:
# Gaussian kernel, R^2
M = landmarks(2)

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(2,N)

In [ ]:
# Exponential kernel, R^2
M = landmarks(2,kernel='K1')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(2,N)

In [ ]:
# Exponential kernel, R^2
M = landmarks(2,kernel='K2')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(2,N)

In [ ]:
# Exponential kernel, R^2
M = landmarks(2,kernel='K3')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(2,N)

In [ ]:
# Exponential kernel, R^2
M = landmarks(2,kernel='K4')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(2,N)

In [ ]:
# Gaussian kernel, R^3
M = landmarks(2)

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(3,N)

In [ ]:
# Exponential kernel, R^3
M = landmarks(2,kernel='K1')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(3,N)

In [ ]:
# Exponential kernel, R^3
M = landmarks(2,kernel='K2')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(3,N)

In [ ]:
# Exponential kernel, R^3
M = landmarks(2,kernel='K3')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(3,N)

In [ ]:
# Exponential kernel, R^3
M = landmarks(2,kernel='K4')

from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

simBrownianN2(3,N)